----
<img src="../../files/lseg.svg" class="rft-examples-logo" width="20%" style="vertical-align: top;">

# Data Library for Python

----

## Content layer - Pricing stream - How to update a DataFrame with streaming events
This notebook demonstrates how to use a Pricing stream objects with events to update a Pandas DataFrame in real-time streaming data. Using a Princing streams that way allows your application to have at its own in memory representation (a Pandas DataFrame in this example) that is kept updated with the latest streaming values received from the LSEG Data Platform.

#### Learn more

To learn more about the LSEG Data Library for Python please join the LSEG Developer Community. By [registering](https://developers.lseg.com/iam/register) and [logging](https://developers.lseg.com/content/devportal/en_us/initCookie.html) into the LSEG Developer Community portal you will have free access to a number of learning materials like 
 [Quick Start guides](https://developers.lseg.com/en/api-catalog/lseg-data-platform/lseg-data-library-for-python/quick-start), 
 [Tutorials](https://developers.lseg.com/en/api-catalog/lseg-data-platform/lseg-data-library-for-python/tutorials), 
 [Documentation](https://developers.lseg.com/en/api-catalog/lseg-data-platform/lseg-data-library-for-python/documentation)
 and much more.

#### Getting Help and Support

If you have any questions regarding using the API, please post them on 
the [Data Library Q&A Forum](https://community.developers.refinitiv.com/smart-spaces/521/index.html). 
The LSEG Developer Community will be happy to help. 

----

## Some Imports to start with

In [1]:
import lseg.data as ld
from pandas import DataFrame
from IPython.display import display, clear_output

## Open the data session

The open_session() function creates and open sessions based on the information contained in the lseg-data.config.json configuration file. Please edit this file to set the session type and other parameters required for the session you want to open.

In [2]:
ld.open_session()

<lseg.data.session.Definition object at 0x168a1c490 {name='workspace'}>

## Retrieve data

### Define callbacks to receive streaming data

In [3]:
df = DataFrame()

def create_dataframe(streaming_prices):
    global df
    snapshot = streaming_prices.get_snapshot()
    field_names = snapshot.columns[1:]
    instrument_names = snapshot['Instrument'].values
    df = DataFrame(index=instrument_names, columns=field_names)
    for price in streaming_prices:
        for field_name, field_value in price.get_fields().items():
            df.at[price.name, field_name] = field_value
    display(df)


def update_dataframe(fields, instrument_name, streaming_prices):
    global df
    clear_output(wait=True)
    for field_name, field_value in fields.items():
        df.at[instrument_name, field_name] = field_value
    display(df)

### Create a stream and open it

In [4]:
stream = ld.content.pricing.Definition(
    universe=['EUR=', 'GBP=', 'JPY=', 'CAD='],
    fields=['BID', 'ASK']
).get_stream()


stream.on_update(update_dataframe)
stream.on_complete(create_dataframe)

stream.open()

,ASK,BID
EUR=,1.1117,1.1116
GBP=,1.3171,1.317
JPY=,142.59,142.58
CAD=,1.3505,1.3503


### Close the stream

In [5]:
stream.close()

<OpenState.Closed: 'Closed'>

## Close the session

In [6]:
ld.close_session()